In [0]:
import os

GALAXY = 'm31'
GALAXYFULLNAME = 'm31'

DEBUG = False

CUT_NB = True
KEEP_BLUE = False

DATA_DIR = os.environ['PFS_DATA_DIR']

OBS_PATH = f'{DATA_DIR}/data/cmdfit/m31/M31Catalog_forPFS.csv'

# SIM_PATH = '/datascope/subaru/user/dobos/cmdfit/run/fornax/sim/mix_bin_250k_001/sample.h5'
# SIM_PATH = f'{DATA_DIR}/data/cmdfit/run/scl/sim/mix_bin_100k_5/sample.h5'
SIM_PATH = f'{DATA_DIR}/data/cmdfit/run/m31/sim/nobin_chab_250k_001/sample.h5'

OUTPUT_PATH = f'{DATA_DIR}/data/targeting/{GALAXYFULLNAME}'

# Create the probability map

Load a CMD simulation, merge foreground and member stellar populations and generate probability map. The script also updates the population weights to match the data.

The simulation contain an equal number of stars for each population and vector of population weights. An indicator variable is also generated based on the weights to select a random sample from the simulated stars that follows the population weights. The probability map is calculated from all stars and weighted by the population weights.

The population weights of the MW foreground come from Galaxia but they often seem off. Until the population parameters can be automatically estimated by the Bayesian code, we update these weights to better match the observed CMD. This is done by eye.

Each population can consist of two sub-populations: single stars and binary stars. If we are only interested in the dSph membership, and the dSph is modeled as a mixture of multiple populations, all sub-populations belonging to the dSph should be merged. The simulations are configured such a way that the dSph member populations are always at the end when ordered by index but their number can vary.

In [0]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from scipy.special import logsumexp
from scipy.interpolate import interp1d

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

## Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.m31 import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

## Load observations

In [0]:
obs = SubaruHSC.text_observation_reader_m31().read(OBS_PATH)
obs.data['targetid'] = np.arange(len(obs.data))+1
obs.data.head()

In [0]:
# Plot the observations

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_catalog(axs[0], obs, observed=True)
ccd.plot_catalog(axs[1], obs, observed=True)

# Load simulation

In [0]:
from pfs.ga.targeting.io import Hdf5SimulationReader

In [0]:
r = Hdf5SimulationReader()
cm = {}

# Include this for loop for simulations made with older isochrone tables that have different column names
for prefix in ['', 'obs_', 'err_', 'flux_', 'obs_flux_', 'err_flux_', 'counts_', 'obs_counts_', 'err_counts_']:
    cm[prefix + 'hsc_g2'] = prefix + 'hsc_g'
    cm[prefix + 'hsc_i2'] = prefix + 'hsc_i'

r.column_mapping = cm
r.append_photometry(SubaruHSC.photometry())
sim = r.read(SIM_PATH)

In [0]:
for k in sim.data.keys():
    print(k, sim.data[k].shape)

In [0]:
s = np.s_[::10]

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_simulation(axs[0], sim, s=s, size=0.05)
ccd.plot_simulation(axs[1], sim, s=s, size=0.05)

f.tight_layout()

# Apply selections

Apply the color and magnitude cuts defined in the galaxy class.

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling, MagnitudeSelection, ColorSelection, LinearSelection

In [0]:
# Plot the observations

mask = galaxy.get_selection_mask(obs, observed=True, nb=CUT_NB, blue=KEEP_BLUE)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_catalog(axs[0], obs, observed=True)
cmd.plot_catalog(axs[1], obs, observed=True, mask=mask)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

ccd.plot_catalog(axs[0], obs, observed=True)
ccd.plot_catalog(axs[1], obs, observed=True, mask=mask)

In [0]:
# Plot the simulations

mask = galaxy.get_selection_mask(sim, observed=True, nb=CUT_NB, blue=KEEP_BLUE)
print(mask.shape)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_catalog(axs[0], sim, observed=True)
cmd.plot_catalog(axs[1], sim, observed=True, mask=mask)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

ccd.plot_catalog(axs[0], sim, observed=True)
ccd.plot_catalog(axs[1], sim, observed=True, mask=mask)

# Update the population weights

This is basically just manually scaling the Galaxia MW population weights until it 
matches the observations.

In [0]:
# This is for Ursa Minor
if GALAXY == 'umi':
    # Original weights
    print('data.w', sim.data['w'])

    w = np.bincount(sim.data['g']) / sim.data['g'].shape
    print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

    # New weights, boost thick disk and halo
    w1 = np.r_[w[:-2] / 0.4 * 0.5, w[-2:] / 0.6 * 0.5]
    w1[2:4] *= 100
    # w1[0:6] *= 3  # thin disk 1-3
    # w1[4:6] *= 1.2  # thin disk 3
    w1[6:8] *= 15  # thick disk
    w1[8:10] *= 28   # halo
    ##### good for histograms w1[10:12] *= 18  # dSph
    ##### good for ghost plot
    w1[10:12] *= 50  # dSph
    w1 /= w1.sum()
    print('w1', w1, np.sum(w1[:-2]), np.sum(w1[-2:]))

    # New categories
    g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
    print('g1', g1.shape)

    # Verify new categories
    w2 = np.bincount(g1) / g1.shape
    print('w2', w2, np.sum(w2[:-2]), np.sum(w2[-2:]))
elif GALAXY == 'fornax':
    # Original weights
    print('data.w', sim.data['w'])

    w = np.bincount(sim.data['g']) / sim.data['g'].shape
    print('w', w.shape, w, np.sum(w[:-2]), np.sum(w[-2:]))

    w1 = w.copy()
    w1[:-6] * 0.7       # MW foreground
    w1[-6:-4] *= 2.0    # broad RGB population
    w1[-4:-2] *= 0.3    # old RGB population
    w1[-2:] *= 1.0      # member MS population
    w1 /= w1.sum()

    g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
    print('g1', g1.shape)

    # Verify new categories
    w2 = np.bincount(g1) / g1.shape
    print('w2', w2.shape, w2, np.sum(w2[:-2]), np.sum(w2[-2:]))
else:
    w = np.bincount(sim.data['g']) / sim.data['g'].shape
    print('w', w.shape, w, np.sum(w[:-2]), np.sum(w[-2:]))
    w1 = w.copy()
    g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
    print('g1', g1.shape)
    w2 = np.bincount(g1) / g1.shape
    print('w2', w2.shape, w2, np.sum(w2[:-2]), np.sum(w2[-2:]))

In [0]:
# Number of objects inside cuts
mask = galaxy.get_selection_mask(obs, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
n_obs = mask.sum()
print('obs', n_obs)

mask = galaxy.get_selection_mask(sim, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
mask = sim.apply_categories(mask, g=g1)
n_sim = mask.sum()
print('sim', n_sim)

n_sim / n_obs

In [0]:
f, axs = plt.subplots(2, 3, figsize=(6, 6), dpi=120)

s = np.s_[::1]

mask = galaxy.get_selection_mask(obs, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
cmd.plot_observation(axs[0, 0], obs, size=0.05, mask=mask, s=s)
ccd.plot_observation(axs[1, 0], obs, size=0.05, mask=mask, s=s)
axs[0, 0].set_title('OBS')

s = np.s_[::3]

mask = galaxy.get_selection_mask(sim, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
# mask = sim.apply_categories(mask, g=g1)
cmd.plot_simulation(axs[0, 1], sim, observed=True, apply_categories=True, mask=mask, g=g1, s=s, size=0.05)
ccd.plot_simulation(axs[1, 1], sim, observed=True, apply_categories=True, mask=mask, g=g1, s=s, size=0.05)
axs[0, 1].set_title('SIM updated weights')

mask = galaxy.get_selection_mask(sim, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
# mask = sim.apply_categories(mask, g=sim.data['g'])
cmd.plot_simulation(axs[0, 2], sim, observed=True, apply_categories=True, mask=mask, g=sim.data['g'], s=s, size=0.05)
ccd.plot_simulation(axs[1, 2], sim, observed=True, apply_categories=True, mask=mask, g=sim.data['g'], s=s, size=0.05)
axs[0, 2].set_title('SIM original weights')

for ax in axs.flatten():
    ax.grid()
    ax.set_xlim(-1, 2.2)

f.tight_layout()

## Plot color histograms

In [0]:
def plot_histogram(ax, obs, sim, axis, bins, plot_populations=True):
    ((x, x_err),) = obs.get_diagram_values([axis], observed=True)
    mask = galaxy.get_selection_mask(obs, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
    hist, bins = np.histogram(x[mask], bins=bins, density=True)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, lw=1, label='OBS')
    print(x.min(), x.max())

    ((x, x_err),) = sim.get_diagram_values([axis], observed=True)
    mask = galaxy.get_selection_mask(sim, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
    mask = sim.apply_categories(mask, g=g1)
    x = sim.apply_categories(x, g=g1)
    hist, bins = np.histogram(x[mask], bins=bins, density=True)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, lw=1, label='SIM')
    
    if plot_populations:
        for i, name in enumerate(['thin1', 'thin2', 'thin3', 'thick', 'halo', 'dSph']):
        # for i, name in enumerate(['thin1', 'thin2', 'thin3', 'thick', 'halo', 'dSph1', 'dSph2', 'dSph3']):
            hist, bins = np.histogram(x[mask][(g1[mask[:,0]] == 2 * i) | (g1[mask[:,0]] == 2 * i + 1)], bins=bins, density=True)
            ax.step(0.5 * (bins[1:] + bins[:-1]), (w1[2 * i] + w1[2 * i + 1]) * hist, lw=0.5, label=name)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.5, 2.4), dpi=240)

plot_histogram(ax, obs, sim, cmd.axes[0], bins=np.linspace(-1.0, 2.0, 100))

ax.set_xlim(-1, 2.2)
ax.set_xlabel('HSC $g - i$')
ax.legend()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.5, 2.4), dpi=240)

plot_histogram(ax, obs, sim, cmd.axes[1], bins=np.linspace(16, 23, 100))

ax.set_xlabel('HSC $g$')
ax.legend()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.5, 2.4), dpi=240)

plot_histogram(ax, obs, sim, MagnitudeAxis(hsc.magnitudes['i']), bins=np.linspace(16, 24, 100))

ax.set_xlabel('HSC $i$')
ax.legend()

# Create probability map

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling

In [0]:
# Ursa Minor dwarf
if GALAXY == 'umi':
    # Original weights
    print('data.w', sim.data['w'])

    w = np.bincount(sim.data['g']) / sim.data['g'].shape
    print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

    # New weights, boost thick disk and halo
    w1 = np.r_[w[:-2] / 0.4 * 0.5, w[-2:] / 0.6 * 0.5]
    w1[2:4] *= 100
    # w1[0:6] *= 3  # thin disk 1-3
    # w1[4:6] *= 1.2  # thin disk 3
    w1[6:8] *= 15  # thick disk
    w1[8:10] *= 28   # halo
    ##### good for histograms w1[10:12] *= 18  # dSph
    ##### good for ghost plot
    w1[10:12] *= 50  # dSph
    w1 /= w1.sum()
    print('w1', w1, np.sum(w1[:-2]), np.sum(w1[-2:]))

    # New categories
    g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
    print('g1', g1.shape)

    # Verify new categories
    w2 = np.bincount(g1) / g1.shape
    print('w2', w2, np.sum(w2[:-2]), np.sum(w2[-2:]))
elif GALAXY == 'fornax':
    # Original weights
    print('data.w', sim.data['w'])

    w = np.bincount(sim.data['g']) / sim.data['g'].shape
    print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

    w1 = w.copy()
    w1[:-6] * 0.7       # MW foreground
    w1[-6:-4] *= 2.0    # broad RGB population
    w1[-4:-2] *= 0.3    # old RGB population
    w1[-2:] *= 1.0      # member MS population
    w1 /= w1.sum()

    g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
    print('g1', g1.shape)

    # Verify new categories
    w2 = np.bincount(g1) / g1.shape
    print('w2', w2, np.sum(w2[:-2]), np.sum(w2[-2:]))
else:
    w = np.bincount(sim.data['g']) / sim.data['g'].shape
    print('w', w.shape, w, np.sum(w[:-2]), np.sum(w[-2:]))
    w1 = w.copy()
    g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
    print('g1', g1.shape)
    w2 = np.bincount(g1) / g1.shape
    print('w2', w2.shape, w2, np.sum(w2[:-2]), np.sum(w2[-2:]))

In [0]:
# The simulation has 10 + 2 * N sub-populations (binaries treated separately for each)
# 10 is for 5 MW (3 for the thin disk + 1 thick disk + 1 halo)
# Merge foreground populations and members + member binaries when creating the map

if GALAXY == 'umi':
    extents = [[0.1, 2.0], [17.0, 23.5]]
elif GALAXY == 'fornax':
    extents = [[-0.75, 2.0], [17.0, 23.5]]
else:
    extents = [[0.1, 2.5], [17.0, 23.5]]

mask = galaxy.get_selection_mask(sim, nb=CUT_NB, blue=KEEP_BLUE, observed=True)
# mask = sim.apply_categories(mask, g=g1)

pmap = ProbabilityMap(cmd.axes)
pmap.from_simulation(sim, bins=[100, 100], extents=extents,
    merge_list=[np.s_[:5], np.s_[5:]], population_weights=w1, observed=True, mask=mask)
pmap.maximum_filter()

In [0]:
pmap.extents

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

l0 = cmd.plot_probability_map(axs[0], pmap, 0)
l1 = cmd.plot_probability_map(axs[1], pmap, 1)

f.tight_layout()

In [0]:
# Save probability maps

fn = 'pmap'
if CUT_NB:
    fn += '_nb'
if KEEP_BLUE:
    fn += '_blue'
fn += '.h5'

fn = os.path.join(OUTPUT_PATH, fn)
pmap.save(fn)

fn

# Membership probability based on the map

In [0]:
lp_member, mask_member = pmap.lookup_lp_member(obs)

lp_member.shape, np.isnan(lp_member).sum(), np.isnan(lp_member[mask_member]).sum(), mask_member.shape, mask_member.sum()

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, c=lp_member[...,0])
ccd.plot_observation(axs[1], obs, c=lp_member[...,0])

f.tight_layout()